In [6]:
from dotenv import load_dotenv
load_dotenv()

import weaviate
from weaviate.classes.init import Auth

import os
# Best practice: store your credentials in environment variables
weaviate_url = os.environ["WEAVIATE_URL"]
weaviate_api_key = os.environ["WEAVIATE_API_KEY"]

# Connect to Weaviate Cloud
client = weaviate.connect_to_weaviate_cloud(
    cluster_url=weaviate_url,
    auth_credentials=Auth.api_key(weaviate_api_key),
)

print(client.is_ready())

True


In [7]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

embed_model = OpenAIEmbedding(model="text-embedding-3-large")

Settings.embed_model = embed_model

embeddings = embed_model.get_text_embedding(
    "OpenAI new Embeddings models is great."
)

print(embeddings[:5])

print("\n")

print(len(embeddings))

[-0.007708967197686434, 0.039421651512384415, -0.017059333622455597, -0.019977377727627754, 0.02456488087773323]


3072


In [14]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SimpleNodeParser

TESLA_DOC = './.data/Tesla Cybertruck Owners Manual.md'
docs = SimpleDirectoryReader(input_files=[TESLA_DOC]).load_data()

parser = SimpleNodeParser()
nodes = parser.get_nodes_from_documents(docs)
print(f"Loaded {len(nodes):,} nodes from '{TESLA_DOC}'")

Loaded 1,281 nodes from './.data/Tesla Cybertruck Owners Manual.md'


In [16]:
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.vector_stores.weaviate import WeaviateVectorStore

COLLECTION_NAME = "TeslaCybertruckOwnersManual"

vector_store = WeaviateVectorStore(weaviate_client = client, index_name=COLLECTION_NAME)
storage_context = StorageContext.from_defaults(vector_store = vector_store)
index = VectorStoreIndex(nodes, embed_model=embed_model, storage_context = storage_context)
print(f"setup index: {index.index_id} for {COLLECTION_NAME}")

setup index: 82012b44-b6a2-4551-a926-d6bce1248b18 for TeslaCybertruckOwnersManual


In [17]:
query_engine = index.as_query_engine()
response = query_engine.query("How can I maximize my charge?")
print(response)

client.close()

To maximize your charge, you should follow these tips: slow down your driving and avoid rapid acceleration, use regenerative braking when slowing down, limit the use of resources like heating and air conditioning, precondition your vehicle using the mobile app, set a schedule for preconditioning and charging, ensure proper wheel alignment and tire pressure, install wheel covers to reduce wind resistance, remove unnecessary cargo to lighten the load, fully raise windows and close the tonneau cover, disable energy-consuming features when not needed, and keep the vehicle plugged in when idle to prevent excessive energy consumption.
